*Note that this notebook should be run with a dedicated compute cluster with the ML Runtime.  It was developed using DBR 16.2 ML originally.  Updates to this notebook were written using ML DBR 16.4LTS and the MlFlow 3 Preview.* 

# Redox Request SQL UDF 
***

The Redox API's authentication methods require the use of pyjwt plus several public and private keys and ids including a PEM file.  At time of writing the Unity Catalog http connection does not support this type of authentication, therefore to create a SQL UDF for interacting with this API (for agent frameworks or general analytics and Business Intelligence) requires the use of Mosaic Model Serving of a Unity Catalog model registered using MLFlow's pyfunc flavor.  

This notebook achieves the following:  

* Demonstrates the normal usuage of the Redox API accelerator that may be used for posting back FHIR compliant bundles to Redox, or retrieving real time data directly from the EMR.  More information about this Databricks Industry Acclerator may be found here:  [https://github.com/databricks-industry-solutions/redox-ehr-api](https://github.com/databricks-industry-solutions/redox-ehr-api)  
* Uses MFLow's Pyfunc Flavor to create a model function that can be used to authenticate and call the Redox API directly in any setting, including with Mosiac Model Serving's serverless environment.  
* Registers the model in Unity Catalog and adds the @latest alais to the latest run.  
* Uses the Databricks Python SDK to serve the Model with Mosiac Model Serving, and the AI Gateway Preview for inference tables, usage and cost monitoring in the System Catalog's serving schema.  

***
## Initial Setup

In [0]:
dbutils.widgets.dropdown("catalog_use", "redox", ["redox", "fhir_workshop"])
dbutils.widgets.dropdown("schema_use", "hls_webinar_fy25q4", ["hls_webinar_fy25q4", "synthea"])
dbutils.widgets.text("databricks_secret_scope", "redox-field-eng", "Secret Scope for the Redox Auth Keys and Ids")

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
%restart_python

In [0]:
%pip install git+https://github.com/databricks-industry-solutions/redox-ehr-api

In [0]:
%pip install mlflow --upgrade --pre

In [0]:
%restart_python

## Standard Usage 
***

In [0]:
databricks_secret_scope = dbutils.widgets.get("databricks_secret_scope")

redox_private_key = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_private_key")
redox_client_id = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_client_id")
redox_source_id = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_source_id")
redox_public_kid = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_public_kid")

print(f""" 
      redox_private_key: {redox_private_key}
      redox_client_id: {redox_client_id}
      redox_source_id: {redox_source_id}
      redox_public_kid: {redox_public_kid}
""")

In [0]:
import json

redox_auth_json = f"""
{{
  "kty": "RSA",
  "kid": "{redox_public_kid}",
  "alg": "RS384",
  "use": "sig"
}}
"""

json.loads(redox_auth_json)

In [0]:
from redoxwrite.auth import * 
from redoxwrite.endpoint import *

In [0]:
auth = RedoxApiAuth(
  redox_client_id
  ,redox_private_key
  ,redox_auth_json
  ,redox_source_id
)
print("Is connection successful? " + str(auth.can_connect()))

In [0]:
#All Redox FHIR request URLs start with this base: https://api.redoxengine.com/fhir/R4/[organization-name]/[environment-type]/
redox_base_url = 'https://api.redoxengine.com/fhir/R4/redox-fhir-sandbox/Development/'

rapi = RedoxApiRequest(auth, base_url = redox_base_url)

In [0]:
#creating an observation for remaining length of 4 day stay at a hospital 
observation = """
{
   "resourceType":"Bundle",
   "entry":[
      {
         "resource":{
            "category":[
               {
                  "coding":[
                     {
                        "code":"survey",
                        "display":"Survey",
                        "system":"http://terminology.hl7.org/CodeSystem/observation-category"
                     }
                  ]
               }
            ],
            "code":{
               "coding":[
                  {
                     "code":"78033-8",
                     "display":"Remaining Hospital Stay",
                     "system":"http://loinc.org"
                  }
               ],
               "text":"Remaining Hospital Stay"
            },
            "effectiveDateTime":"2024-01-28T18:06:33.245-05:00",
            "issued":"2024-01-28T18:06:33.245-05:00",
            "resourceType":"Observation",
            "status":"final",
            "valueQuantity":{
               "code":"days",
               "system":"https://www.nubc.org/CodeSystem/RevenueCodes",
               "unit":"days",
               "value":4
            },
            "subject": {
              "reference": "Patient/58117110-ae47-452a-be2c-2d82b3a9e24b"
            },
            "identifier": [
            {
              "system": "urn:databricks",
              "value": "1234567890"
            }
          ]
         }
      },
      {
         "resource":{
           "resourceType": "Patient",
           "identifier": [
            {
              "system": "urn:redox:health-one:MR",
              "value": "0000991458"
            },
            {
              "system": "http://hl7.org/fhir/sid/us-ssn",
              "value": "547-01-9991"
            }
          ]
         }
      }
   ]
}
"""

In [0]:
result = rapi.make_request(
  http_method="post"
  ,resource="Observation"
  ,action="$observation-create"
  ,data=observation
)

In [0]:
import json

print(f"Response Status Code: {result['response']['response_status_code']}")
print("\n")
print(json.dumps(json.loads(result['response']['response_text']), indent=4))

## Using MLFlow to Register the Redox API as a Pyfunc Flavor
***

In [0]:
import mlflow
from mlflow.pyfunc.utils import pyfunc
from os import environ

In [0]:
environ['REDOX_PRIVATE_KEY'] = redox_private_key
environ['REDOX_CLIENT_ID'] = redox_client_id
environ['REDOX_SOURCE_ID'] = redox_source_id
environ['REDOX_PUBLIC_KID'] = redox_public_kid

In [0]:
class RedoxMakeRequest(mlflow.pyfunc.PythonModel):
    def __init__(self, base_url):
        self.base_url = base_url

    def load_context(self, context):
        import redoxwrite.auth
        import redoxwrite.endpoint
        from os import environ
        self.redox_private_key = environ['REDOX_PRIVATE_KEY']
        self.redox_client_id = environ['REDOX_CLIENT_ID']
        self.redox_source_id = environ['REDOX_SOURCE_ID']
        self.redox_public_kid = environ['REDOX_PUBLIC_KID']
        self.redox_auth_json = f"""
                {{
                "kty": "RSA",
                "kid": "{self.redox_public_kid}",
                "alg": "RS384",
                "use": "sig"
                }}
            """
        self.auth = RedoxApiAuth(
                self.redox_client_id
                ,self.redox_private_key
                ,self.redox_auth_json
                ,self.redox_source_id
            )
        self.rapi = RedoxApiRequest(self.auth, base_url = self.base_url)
    
    @pyfunc
    def predict(self, context, model_input, params = None) -> list[str]: 
        results = []
        # Apply the self.rapi.make_request function to each row of the DataFrame
        # Assumes that the model input contains the following columns:
            # http_method, resource, action, data where data is a value FHIR payload
        # Convert Pandas DF to string.
        for row in model_input.itertuples(index=False):
            result = self.rapi.make_request(
                http_method=row.http_method
                ,resource=row.resource
                ,action=row.action
                ,data=row.data
            )
            respone_text = result['response']['response_text']
            results.append(respone_text)
        return results

In [0]:
import pandas as pd

df = pd.DataFrame({"http_method": ["post", "post"], "resource": ["Observation", "DiagnosticReport"], "action": ["$observation-create", "_search"], "data": [observation, "subject=Patient/81c2f5eb-f99f-40c4-b504-59483e6148d7'"]})
df

In [0]:
signature = mlflow.models.infer_signature(df, ["redox_make_request"])
signature

In [0]:
conda_env = mlflow.pyfunc.get_default_conda_env()
conda_env["dependencies"][-1]["pip"].append("git+https://github.com/databricks-industry-solutions/redox-ehr-api")
conda_env["dependencies"][-1]["pip"].append("pyjwt")
conda_env["dependencies"][-1]["pip"].append("cryptography")
print(conda_env)

In [0]:
with mlflow.start_run():

    # Ensure the standard python libraries and the redox-ehr-api classed and methods are installed in the conda environment
    conda_env = mlflow.pyfunc.get_default_conda_env()
    conda_env["dependencies"][-1]["pip"].append("git+https://github.com/databricks-industry-solutions/redox-ehr-api")
    # conda_env["dependencies"][-1]["pip"].append("databricks-sdk")
    conda_env["dependencies"][-1]["pip"].append("pyjwt")
    conda_env["dependencies"][-1]["pip"].append("cryptography")

    input_example = df.iloc[[0]]
    
    model_info = mlflow.pyfunc.log_model(
        name="model"
        ,python_model=RedoxMakeRequest(
            base_url=redox_base_url
        )
        ,signature=signature
        ,conda_env=conda_env
        ,input_example=input_example
    )

    # Log an example of how to use the model
    example_input = df.head(1).to_dict(orient="records")
    mlflow.log_dict(example_input, "input_example.json")

In [0]:
print(f"""
      model_info.model_uri = {model_info.model_uri}
      model_info.model_id = {model_info.model_id}
      model_info.run_id = {model_info.run_id}
""")

In [0]:
logged_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [0]:
# logged_model = f"runs:/{run_id}/model"

# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
logged_model.predict(df.iloc[[0]])

In [0]:
mlflow.set_registry_uri("databricks-uc")

catalog_use = dbutils.widgets.get("catalog_use")
schema_use = dbutils.widgets.get("schema_use")
model_name = "redox_make_request"
full_model_name = f"{catalog_use}.{schema_use}.{model_name}"
full_model_name

In [0]:
# mlflow.register_model(f"runs:/{model_info.run_id}", full_model_name)
mlflow.register_model(model_info.model_uri, full_model_name)

In [0]:
client = mlflow.MlflowClient()

In [0]:
# Search for all versions of the model
model_version_infos = client.search_model_versions(f"name = '{full_model_name}'")

# Find the latest version
latest_version = max([model_version_info.version for model_version_info in model_version_infos])
print(f"""
    The latest version of the model {full_model_name} in Unity Catalog is version {latest_version}.
""")

In [0]:
try:
    prior_model_version = client.get_model_version_by_alias(full_model_name, "latest_version")

    if prior_model_version.version != latest_version:
        client.delete_registered_model_alias(full_model_name, "latest_version")
except:
    pass

# Create an alias for the model version
client.set_registered_model_alias(full_model_name, "latest_version", latest_version)

In [0]:
# Load Unity Catalog model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(f"models:/{full_model_name}@latest_version")

# Predict on a Pandas DataFrame.
loaded_model.predict(df.iloc[[0]])

***
## Mosiac Model Serving

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput, ServingModelWorkloadType, AiGatewayConfig, AiGatewayInferenceTableConfig, AiGatewayUsageTrackingConfig

w = WorkspaceClient()

In [0]:
w.serving_endpoints.create(
    name = model_name
    ,config = EndpointCoreConfigInput(
        name = model_name
        ,served_entities = [
            ServedEntityInput(
                entity_name = full_model_name
                ,entity_version = client.get_model_version_by_alias(full_model_name, "latest_version").version
                ,environment_vars = {
                    'REDOX_PRIVATE_KEY': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_private_key}}"
                    ,'REDOX_CLIENT_ID': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_client_id}}"
                    ,'REDOX_SOURCE_ID': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_source_id}}"
                    ,'REDOX_PUBLIC_KID': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_public_kid}}"
                }
                ,scale_to_zero_enabled = True
                ,workload_size = "Small"
                ,workload_type = ServingModelWorkloadType("CPU")
            )
        ]
    )
    ,ai_gateway = AiGatewayConfig(
        inference_table_config = AiGatewayInferenceTableConfig(
            catalog_name=catalog_use
            ,schema_name=schema_use
            ,table_name_prefix = None
            ,enabled=True
        )
        ,usage_tracking_config = AiGatewayUsageTrackingConfig(
            enabled=True
        )
    )
)